<a href="https://colab.research.google.com/github/DAEN690-S22-OA/OceanicAirspaceProject/blob/main/VerticalSeparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data pre-processing will happen here. 

In [ ]:
#Installing the required libraries
!pip install geopy
!pip install pandasql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 32 kB/s 
     |████████████████████████████████| 199 kB 63.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=e297d16c17b3168f95b42508667dc387ecb0a52be6c9cc76e7df55f66dc0fadb
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 60 kB/s 
  Created wheel for spark: filename=spark-0.2.1-py3-none-any.whl size=58762 sha256=36cbd49ffe3040a5e550339ecb66b00c77e767522f7fec144d89266d440d3021
  Stored in directory: /root/.cache/pip/wheels/4e/0e/f1/164619f9920fb447d294afaae11a7715bd442ded7225953d72
Successfully built spark
Looking in indexes: https://pypi.org/simple, https://us-p

In [ ]:
#Importing the required libraries
import pandas as pd
import pandasql as ps
from pyspark.sql import SparkSession
import os
import sys
import re
from datetime import datetime
from geopy.distance import geodesic as GD

In [ ]:
#Upload the data file after pre-processing (only the selected attribues and only above level 240)
from google.colab import files
uploadedfile = files.upload()


Saving airspaceData.csv to airspaceData.csv


In [ ]:
import io
airspaceDF = pd.read_csv(io.BytesIO(uploadedfile['airspaceData.csv']))

In [ ]:
print(airspaceDF)

                        DateTime   Hour   Minute   Second   microSecond  \
0        2021-12-24 23:59:59.992     23       59       59        992000   
1        2021-12-24 20:54:11.141     20       54       11        141000   
2        2021-12-24 12:36:02.773     12       36        2        773000   
3        2021-12-24 04:13:17.836      4       13       17        836000   
4        2021-12-24 23:59:59.984     23       59       59        984000   
...                          ...    ...      ...      ...           ...   
3303983  2021-12-24 12:36:02.859     12       36        2        859000   
3303984  2021-12-24 12:36:02.852     12       36        2        852000   
3303985  2021-12-24 12:36:02.805     12       36        2        805000   
3303986  2021-12-24 12:36:02.805     12       36        2        805000   
3303987  2021-12-24 12:36:02.789     12       36        2        789000   

          Latitude   Longitude   Height   TargetID    SelectedHeading  \
0        21.325208 -158.05

In [ ]:
#Some column names may be imported with leading spaces. This will remove that
airspaceDF.columns = [x.strip() for x in airspaceDF.columns]

print(airspaceDF.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3303988 entries, 0 to 3303987
Data columns (total 12 columns):
 #   Column           Dtype  
---  ------           -----  
 0   DateTime         object 
 1   Hour             int64  
 2   Minute           int64  
 3   Second           int64  
 4   microSecond      int64  
 5   Latitude         float64
 6   Longitude        float64
 7   Height           float64
 8   TargetID         object 
 9   SelectedHeading  float64
 10  TargetAddress    object 
 11  TrackNumber      float64
dtypes: float64(5), int64(4), object(3)
memory usage: 302.5+ MB
None


In [ ]:
#create SQL query for flights between the start and end time
sql1 = "SELECT * FROM airspaceDF WHERE Hour = 23 and Minute = 59 ORDER BY TargetID, Second"

#Run query and store results
records2359 = ps.sqldf(sql1, locals())

In [ ]:
#Export the file with just the 23:59 data to make it easier to pick up where we left off
from google.colab import files

records2359.to_csv('records2359_1224.csv', encoding = 'utf-8-sig') 
files.download('records2359_1224.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**START HERE WITH records2359_1224.csv FILE**

In [ ]:
#Upload the file with the 23:59 data and save it as a dataframe named records2359

#Upload the records2359_1224.csv data file
from google.colab import files
uploadedfile = files.upload()

import io
records2359 = pd.read_csv(io.BytesIO(uploadedfile['records2359_1224.csv']))

Saving records2359_1224.csv to records2359_1224.csv


In [ ]:
#Remove flights without a flight name or altitude
records2359 = records2359.dropna(subset=['TargetID'])
records2359 = records2359.dropna(subset=['Height'])

records2359.head(5)

,Unnamed: 0,DateTime,Hour,Minute,Second,microSecond,Latitude,Longitude,Height,TargetID,SelectedHeading,TargetAddress,TrackNumber
16,16,59:01.9,23,59,1,875000,21.085836,-157.526245,15400.0,AAH24,107.578125,A32F17,1234.0
17,17,59:06.8,23,59,6,750000,21.081926,-157.517598,15400.0,AAH24,107.578125,A32F17,1234.0
18,18,59:10.9,23,59,10,875000,21.078644,-157.510337,15400.0,AAH24,107.578125,A32F17,1234.0
19,19,59:15.1,23,59,15,102000,21.075211,-157.502747,15425.0,AAH24,107.578125,A32F17,1234.0
20,20,59:19.7,23,59,19,695000,21.071594,-157.494807,15400.0,AAH24,107.578125,A32F17,1234.0


In [ ]:
#Get the first entry for this minute of time
sql2 = "SELECT *, min(Second) FROM records2359 GROUP BY TargetID"

#Run query and store results
firstEntryDF = ps.sqldf(sql2, locals())
del firstEntryDF['min(Second)']

firstEntryDF.head(5)

,Unnamed: 0,DateTime,Hour,Minute,Second,microSecond,Latitude,Longitude,Height,TargetID,SelectedHeading,TargetAddress,TrackNumber
0,16,59:01.9,23,59,1,875000,21.085836,-157.526245,15400.0,AAH24,107.578125,A32F17,1234.0
1,30,59:01.5,23,59,1,508000,19.976543,-155.128401,2625.0,AAH504,130.078125,A323F2,1272.0
2,43,59:03.1,23,59,3,109000,25.290344,-147.790578,38775.0,AAL119,232.031250,AB6AFA,1024.0
3,56,59:02.3,23,59,2,273000,42.300705,-132.887941,38200.0,AAL128,99.843750,AB7613,1024.0
4,69,59:01.4,23,59,1,438000,33.262985,-123.632941,30500.0,AAL143,0.000000,A4FBAC,1024.0


In [ ]:
#Create tables to store proximity
sql3 = "SELECT * FROM firstEntryDF ORDER BY Latitude"
sql4 = "SELECT * FROM firstEntryDF ORDER BY Longitude"

#Run query and store results
LatitudeOrderDF = ps.sqldf(sql3, locals())
LongitudeOrderDF = ps.sqldf(sql4, locals())


In [ ]:
LatitudeOrderDF.head()

,Unnamed: 0,DateTime,Hour,Minute,Second,microSecond,Latitude,Longitude,Height,TargetID,SelectedHeading,TargetAddress,TrackNumber
0,1100,59:04.0,23,59,4,47000,0.190538,-175.769980,37350.0,FDX77,210.937500,ABE1EB,1024.0
1,1800,59:00.6,23,59,0,625000,6.845460,-163.240356,36000.0,MGE600,NaN,AC3C99,1484.0
2,659,59:04.0,23,59,3,977000,11.171885,178.954757,37225.0,ATN440,51.328125,AA21BF,1024.0
3,1366,59:01.5,23,59,1,453000,11.610489,-166.661035,42300.0,HAL451,NaN,A45233,1024.0
4,2578,59:04.8,23,59,4,836000,12.478409,142.616035,40650.0,SIA8890,258.750000,76B44A,1997.0


In [ ]:
LongitudeOrderDF.head()

,DateTime,Hour,Minute,Second,microSecond,Latitude,Longitude,Height,TargetID,SelectedHeading,TargetAddress,TrackNumber
0,2021-12-24 23:59:03.648,23,59,3,648000,17.969330,-176.990517,38025.0,MGE300,NaN,A9228A,1088.0
1,2021-12-24 23:59:04.047,23,59,4,47000,0.190538,-175.769980,37350.0,FDX77,210.937500,ABE1EB,1024.0
2,2021-12-24 23:59:02.883,23,59,2,883000,33.536041,-175.560315,40950.0,UAL903,288.281250,AA9321,1488.0
3,2021-12-24 23:59:00.430,23,59,0,430000,49.627487,-174.973249,34575.0,UAL893,265.078125,A31A73,1024.0
4,2021-12-24 23:59:00.109,23,59,0,109000,47.560684,-172.337723,40800.0,AAL9726,277.031250,AB5257,1024.0


In [ ]:
#Calculate the distance of the points closest to each other by longitidue and latitude

latInterest1 = []
latInterest2 = []
latDistance = []

longInterest1 = []
longInterest2 = []
longDistance = []

counter1 = 0
counter2 = 1

while counter2 < len(LongitudeOrderDF):
  #Create the coordinates that Geopy will ingest
  point0 = (LongitudeOrderDF['Latitude'][counter1], LongitudeOrderDF['Longitude'][counter1])
  point1 = (LongitudeOrderDF['Latitude'][counter2], LongitudeOrderDF['Longitude'][counter2])

  #Calculate the distance between the two points
  distance = GD(point0,point1).km

  #Assuming the distance threshold is 5KM
  if distance < 5:
    longInterest1.append(counter1)
    longInterest2.append(counter2)
    longDistance.append(distance)

  #Increase the count
  counter1 = counter1 + 1
  counter2 = counter2 + 1

print(longInterest1)
print(longInterest2)
print(longDistance)

print('')
print('---')
print('')

counter1 = 0
counter2 = 1

while counter2 < len(LatitudeOrderDF):
  #Create the coordinates that Geopy will ingest
  point0 = (LatitudeOrderDF['Latitude'][counter1], LatitudeOrderDF['Longitude'][counter1])
  point1 = (LatitudeOrderDF['Latitude'][counter2], LatitudeOrderDF['Longitude'][counter2])

  #Calculate the distance between the two points
  distance = GD(point0,point1).km

  #Assuming the distance threshold is 5KM
  if distance < 5:
    latInterest1.append(counter1)
    latInterest2.append(counter2)
    latDistance.append(distance)

  #Increase the count
  counter1 = counter1 + 1
  counter2 = counter2 + 1

print(latInterest1)
print(latInterest2)
print(latDistance)

[21, 24, 25, 26, 30, 31, 32, 41, 48, 49, 50, 71, 80, 103, 127, 138, 195]
[22, 25, 26, 27, 31, 32, 33, 42, 49, 50, 51, 72, 81, 104, 128, 139, 196]
[3.2555790800323257, 2.1313499546741426, 1.434996274386774, 1.7472435861987152, 0.921809942353758, 2.3903466792274917, 4.947650052358498, 4.126299892974009, 2.546333855157941, 1.4134608909271353, 4.886132946915177, 1.496699176920748, 2.3755526867640224, 3.080109603682967, 2.8429956465435025, 3.18890544955417, 4.048862553398835]

---

[10, 27, 49, 50, 58, 80, 88, 90, 91, 92, 132, 143, 207]
[11, 28, 50, 51, 59, 81, 89, 91, 92, 93, 133, 144, 208]
[3.080109603682967, 2.3755526867640224, 4.4389653201839465, 2.3758460203815437, 1.4134608909271353, 0.921809942353758, 3.2555790800323257, 2.1313499546741426, 1.434996274386774, 1.7472435861987152, 2.8429956465435025, 3.18890544955417, 4.048862553398835]


In [ ]:
#Create a combined list of flights that came within 25 nautical miles of eachother at this time stamp

Resultsdf = pd.DataFrame(columns=['remove1','DateTime1','Hour1','Minute1','Second1','microSecond1','Latitude1','Longitude1','Height1',
                                  'TargetID1', 'SelectedHeading1', 'TargetAddress1', 'remove2', "TrackNumber2", 'DateTime2','Hour2','Minute2',
                                  'Second2','microSecond2','Latitude2','Longitude2','Height2', 'TargetID2', 'SelectedHeading2', 
                                  'TargetAddress2', "TrackNumber2", "Distance"])

print(Resultsdf)

Empty DataFrame
Columns: [remove1, DateTime1, Hour1, Minute1, Second1, microSecond1, Latitude1, Longitude1, Height1, TargetID1, SelectedHeading1, TargetAddress1, remove2, TrackNumber2, DateTime2, Hour2, Minute2, Second2, microSecond2, Latitude2, Longitude2, Height2, TargetID2, SelectedHeading2, TargetAddress2, TrackNumber2, Distance]
Index: []

[0 rows x 27 columns]


In [ ]:
#Adding the values from analyzing Longitude 
listlocation = 0

for x in longInterest1:
  position1 = longInterest1[listlocation]
  position2 = longInterest2[listlocation]
  
  entryValues1 = []

  position = 0
  while position < len(LongitudeOrderDF.columns):
    recordData = LongitudeOrderDF.loc[position1][position]
    entryValues1.append(recordData)
    position = position + 1

  position = 0
  while position < len(LongitudeOrderDF.columns):
    recordData = LongitudeOrderDF.loc[position2][position]
    entryValues1.append(recordData)
    position = position + 1

  entryValues1.append(longDistance[listlocation])
  print(entryValues1)

  Resultsdf.loc[len(Resultsdf.index)] = entryValues1

  listlocation = listlocation + 1

[1922, '59:01.7', 23, 59, 1, 719000, 22.16015624, -159.696317, 2450.0, 'N11QN', nan, 'A02B45', 640.0, 1873, '59:00.2', 23, 59, 0, 234000, 22.16091346, -159.6647645, 1800.0, 'N11QC', nan, 'A02B3B', 640.0, 3.2555790800323257]
[2262, '59:14.4', 23, 59, 14, 414000, 22.20890807, -159.6128208, 975.0, 'N635SB', nan, 'A851E1', 640.0, 2187, '59:02.5', 23, 59, 2, 523000, 22.21609488, -159.5936447, 1175.0, 'N451H', nan, 'A57682', 640.0, 2.1313499546741426]
[2187, '59:02.5', 23, 59, 2, 523000, 22.21609488, -159.5936447, 1175.0, 'N451H', nan, 'A57682', 640.0, 2123, '59:02.9', 23, 59, 2, 898000, 22.22887963, -159.5913697, 1925.0, 'N251H', nan, 'A25BE4', 640.0, 1.434996274386774]
[2123, '59:02.9', 23, 59, 2, 898000, 22.22887963, -159.5913697, 1925.0, 'N251H', nan, 'A25BE4', 640.0, 2438, '59:00.0', 23, 59, 0, 16000, 22.22943837, -159.5744325, 1725.0, 'N963SA', nan, 'AD68DC', nan, 1.7472435861987152]
[2249, '59:01.2', 23, 59, 1, 234000, 21.95539841, -159.3476507, 750.0, 'N633JH', nan, 'A849B1', 640.0, 

In [ ]:
#Adding the values from analyzing Latitude 
listlocation = 0

for x in latInterest1:
  position1 = latInterest1[listlocation]
  position2 = latInterest2[listlocation]
  
  entryValues1 = []

  position = 0
  while position < len(LatitudeOrderDF.columns):
    recordData = LatitudeOrderDF.loc[position1][position]
    entryValues1.append(recordData)
    position = position + 1

  position = 0
  while position < len(LatitudeOrderDF.columns):
    recordData = LatitudeOrderDF.loc[position2][position]
    entryValues1.append(recordData)
    position = position + 1

  entryValues1.append(latDistance[listlocation])

  Resultsdf.loc[len(Resultsdf.index)] = entryValues1

  listlocation = listlocation + 1

In [ ]:
print(Resultsdf)

   remove1 DateTime1 Hour1 Minute1 Second1 microSecond1  Latitude1  \
0     1922   59:01.7    23      59       1       719000  22.160156   
1     2262   59:14.4    23      59      14       414000  22.208908   
2     2187   59:02.5    23      59       2       523000  22.216095   
3     2123   59:02.9    23      59       2       898000  22.228880   
4     2249   59:01.2    23      59       1       234000  21.955398   
5     2030   59:13.7    23      59      13       711000  21.963135   
6     2388   59:08.6    23      59       8       648000  21.984558   
7     3205   59:02.0    23      59       2        23000  21.321670   
8     2900   59:02.6    23      59       2       586000  21.306839   
9     2596   59:23.1    23      59      23       148000  21.322042   
10    2210   59:02.1    23      59       2        78000  21.323066   
11    2410   59:08.7    23      59       8       688000  21.155777   
12    2376   59:08.0    23      59       7       961000  20.869949   
13    2223   59:01.8

In [ ]:
heightDifference = []
potentialLoss1000 = []
potentialLoss400 = []

counter = 0

while counter < len(Resultsdf):
  difference = abs((Resultsdf['Height1'][counter]) - (Resultsdf['Height2'][counter]))
  heightDifference.append(difference)

  if difference <= 1000:
    potentialLoss.append('True')
  else:
    potentialLoss.append('False')

  counter = counter + 1

print(heightDifference)
print(potentialLoss)

[650.0, 200.0, 750.0, 200.0, 800.0, 1400.0, 475.0, 3050.0, 0.0, 275.0, 75.0, 175.0, 1025.0, 575.0, 7950.0, 1925.0, 5000.0, 575.0, 1025.0, 4150.0, 125.0, 275.0, 800.0, 650.0, 200.0, 750.0, 200.0, 7950.0, 1925.0, 5000.0]
['True', 'True', 'True', 'True', 'True', 'False', 'True', 'False', 'True', 'True', 'True', 'True', 'False', 'True', 'False', 'False', 'False', 'True', 'False', 'False', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'False', 'False', 'False']


In [ ]:
Resultsdf['HeightDifference'] = heightDifference
Resultsdf['PotentialLoss'] = potentialLoss

print(Resultsdf)

   remove1 DateTime1 Hour1 Minute1 Second1 microSecond1  Latitude1  \
0     1922   59:01.7    23      59       1       719000  22.160156   
1     2262   59:14.4    23      59      14       414000  22.208908   
2     2187   59:02.5    23      59       2       523000  22.216095   
3     2123   59:02.9    23      59       2       898000  22.228880   
4     2249   59:01.2    23      59       1       234000  21.955398   
5     2030   59:13.7    23      59      13       711000  21.963135   
6     2388   59:08.6    23      59       8       648000  21.984558   
7     3205   59:02.0    23      59       2        23000  21.321670   
8     2900   59:02.6    23      59       2       586000  21.306839   
9     2596   59:23.1    23      59      23       148000  21.322042   
10    2210   59:02.1    23      59       2        78000  21.323066   
11    2410   59:08.7    23      59       8       688000  21.155777   
12    2376   59:08.0    23      59       7       961000  20.869949   
13    2223   59:01.8

In [ ]:
Resultsdf.to_csv('results2359.csv', encoding = 'utf-8-sig') 
files.download('results2359.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Get the first entry for this minute of time
sql6 = "SELECT * FROM Resultsdf WHERE PotentialLoss = 'True' "

#Run query and store results
LossCandidates = ps.sqldf(sql6, locals())

In [ ]:
print(LossCandidates)

    remove1 DateTime1  Hour1  Minute1  Second1  microSecond1  Latitude1  \
0      1922   59:01.7     23       59        1        719000  22.160156   
1      2262   59:14.4     23       59       14        414000  22.208908   
2      2187   59:02.5     23       59        2        523000  22.216095   
3      2123   59:02.9     23       59        2        898000  22.228880   
4      2249   59:01.2     23       59        1        234000  21.955398   
5      2388   59:08.6     23       59        8        648000  21.984558   
6      2900   59:02.6     23       59        2        586000  21.306839   
7      2596   59:23.1     23       59       23        148000  21.322042   
8      2210   59:02.1     23       59        2         78000  21.323066   
9      2410   59:08.7     23       59        8        688000  21.155777   
10     2223   59:01.8     23       59        1        773000  19.419499   
11     1944   59:04.6     23       59        4        633000  19.396179   
12     1911   59:03.1    